# Import dependencies

In [ ]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [1]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from models.BERT_Dropout_Linear import BERT_Dropout_Linear

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [4]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [5]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [6]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [7]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [8]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [9]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_linear.csv'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
best_accuracy = 0.0

In [12]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [13]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [14]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [15]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = BERT_Dropout_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6945077776908875
Epoch: 0/2, Batch: 31/313, Loss: 0.3229191303253174
Epoch: 0/2, Batch: 62/313, Loss: 0.22399303317070007
Epoch: 0/2, Batch: 93/313, Loss: 0.15816020965576172
Epoch: 0/2, Batch: 124/313, Loss: 0.1405986249446869
Epoch: 0/2, Batch: 155/313, Loss: 0.19062122702598572
Epoch: 0/2, Batch: 186/313, Loss: 0.11536185443401337
Epoch: 0/2, Batch: 217/313, Loss: 0.06736935675144196
Epoch: 0/2, Batch: 248/313, Loss: 0.08973481506109238
Epoch: 0/2, Batch: 279/313, Loss: 0.3357014060020447
Epoch: 0/2, Batch: 310/313, Loss: 0.05650490149855614
Epoch: 1/2, Batch: 0/313, Loss: 0.04045473784208298
Epoch: 1/2, Batch: 31/313, Loss: 0.04644361138343811
Epoch: 1/2, Batch: 62/313, Loss: 0.033130381256341934
Epoch: 1/2, Batch: 93/313, Loss: 0.04059445485472679
Epoch: 1/2, Batch: 124/313, Loss: 0.036855779588222504
Epoch: 1/2, Batch: 155/313, Loss: 0.05321710929274559
Epoch: 1/2, Batch: 186/313, Loss: 0.025032205507159233
Epoch: 1/2, Batch: 217/313, Lo

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7134093046188354
Epoch: 0/2, Batch: 31/313, Loss: 0.3314184546470642
Epoch: 0/2, Batch: 62/313, Loss: 0.21228936314582825
Epoch: 0/2, Batch: 93/313, Loss: 0.1513461172580719
Epoch: 0/2, Batch: 124/313, Loss: 0.3745698928833008
Epoch: 0/2, Batch: 155/313, Loss: 0.09636692702770233
Epoch: 0/2, Batch: 186/313, Loss: 0.1138981431722641
Epoch: 0/2, Batch: 217/313, Loss: 0.3704026937484741
Epoch: 0/2, Batch: 248/313, Loss: 0.0727301687002182
Epoch: 0/2, Batch: 279/313, Loss: 0.5548577904701233
Epoch: 0/2, Batch: 310/313, Loss: 0.04964229092001915
Epoch: 1/2, Batch: 0/313, Loss: 0.053741034120321274
Epoch: 1/2, Batch: 31/313, Loss: 0.05015234649181366
Epoch: 1/2, Batch: 62/313, Loss: 0.5765897631645203
Epoch: 1/2, Batch: 93/313, Loss: 0.10813695192337036
Epoch: 1/2, Batch: 124/313, Loss: 0.4221479296684265
Epoch: 1/2, Batch: 155/313, Loss: 0.3398541212081909
Epoch: 1/2, Batch: 186/313, Loss: 0.022787705063819885
Epoch: 1/2, Batch: 217/313, Loss: 0.04

In [16]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.939103,0.939103,0.784160,0.939103,0.683917,0.939103,0.700570,214.968346
1,0.932692,0.932692,0.947937,0.932692,0.663808,0.932692,0.687239,211.561142
2,0.932692,0.932692,0.953672,0.932692,0.662468,0.932692,0.681514,205.901747
3,0.980769,0.980769,0.943135,0.980769,0.914692,0.980769,0.927949,204.181634
4,0.929487,0.929487,0.949062,0.929487,0.704672,0.929487,0.730427,208.950001
5,0.929487,0.929487,0.956338,0.929487,0.660940,0.929487,0.681762,209.558999
6,0.958333,0.958333,0.630906,0.958333,0.646572,0.958333,0.638542,208.894078
7,0.929487,0.929487,0.813442,0.929487,0.736887,0.929487,0.762339,209.888987
8,0.913462,0.913462,0.595703,0.913462,0.631341,0.913462,0.611111,209.368890
9,0.926282,0.926282,0.615683,0.926282,0.644112,0.926282,0.629559,208.753999


In [17]:
results.to_csv(STATS_OUTPUT)